In [1]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.0 MB/s eta 0:00:00


In [2]:
! pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 14.7 MB/s eta 0:00:00


In [19]:
import pandas as pd
import numpy as np
import optuna
import warnings
import pyarrow.parquet as pq
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.feature_selection import SelectPercentile, f_regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.base import BaseEstimator, TransformerMixin
from category_encoders import CatBoostEncoder
from xgboost import XGBClassifier
from scipy.stats import ttest_rel
import seaborn as sns
import matplotlib.pyplot as plt

warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
df = pd.read_parquet('/content/multisim_dataset.parquet')
pd.set_option("display.max_columns", None)  # Show all columns
df.head()

,telephone_number,trf,age,gndr,tenure,age_dev,dev_man,device_os_name,dev_num,is_dualsim,is_featurephone,is_smartphone,simcard_type,region,val2_1,val3_1,val4_1,val5_1,val6_1,val7_1,val8_1,val9_1,val10_1,val11_1,val12_1,val13_1,val14_1,val15_1,val16_1,val17_1,val18_1,val19_1,val20_1,val21_1,val2_2,val3_2,val4_2,val5_2,val6_2,val7_2,val8_2,val9_2,val10_2,val11_2,val12_2,val13_2,val14_2,val15_2,val16_2,val17_2,val18_2,val19_2,val20_2,val21_2,val2_3,val3_3,val4_3,val5_3,val6_3,val7_3,val8_3,val9_3,val10_3,val11_3,val12_3,val13_3,val14_3,val15_3,val16_3,val17_3,val18_3,val19_3,val20_3,val21_3,val2_4,val4_4,val5_4,val6_4,val7_4,val8_4,val9_4,val10_4,val11_4,val12_4,val13_4,val14_4,val15_4,val16_4,val17_4,val18_4,val19_4,val20_4,val21_4,val2_5,val4_5,val5_5,val6_5,val7_5,val8_5,val9_5,val10_5,val11_5,val12_5,val13_5,val14_5,val15_5,val16_5,val17_5,val18_5,val19_5,val20_5,val21_5,val2_6,val4_6,val5_6,val6_6,val7_6,val8_6,val9_6,val10_6,val11_6,val12_6,val13_6,val14_6,val15_6,val16_6,val17_6,val18_6,val19_6,val20_6,val21_6,target
9626,iWBn_13aJO,J,50,M,6957.0,1263,Samsung,Android OS,1,1,0,1,None,GAKH,3.0,3.0,14,7.4,0,0.0,0,0.0,0,0.0,0.000000,0.000000,0.0,0.00,1.12,0.00,0.0,79,14,0,3.00,3.0,13.0,9.0,1.0,0.4,1.0,0.4,0.0,0.0,0.000000,0.000000,0.0,0.14,1.05,0.14,0.00,64.0,13.0,1.0,3.00,3.0,14.0,10.3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.42,0.00,0.00,68.0,14.0,0.0,2.0,4.0,2.8,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,-1.490116e-09,0.00,0.0,39.0,4.0,0.0,3.0,10.0,8.3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.91,0.0,0.0,57.0,10.0,0.0,3.0,7.0,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.56,0.00,0.00,60.0,7.0,0.0,1
34347,8JWaD2wYoQ,M,52,M,1041.0,125,Samsung,Android OS,2,1,0,1,4G,MASALLI,2.0,2.0,6,2.9,0,0.0,0,0.0,0,0.0,176.447991,0.000000,0.0,0.00,0.40,0.00,0.0,118,6,0,1.06,1.0,29.0,21.4,1.0,0.2,0.0,0.0,1.0,0.2,117.043976,116.052454,0.0,0.08,3.12,0.00,0.08,118.0,29.0,1.0,5.71,3.0,26.0,34.4,5.0,1.8,2.0,0.8,2.0,0.8,160.147566,67.178972,0.0,0.40,1.28,0.16,0.16,125.0,26.0,5.0,1.0,1.0,3.2,0.0,0.0,0.0,0.0,0.0,0.0,182.922203,181.783154,0.0,0.00,0.000000e+00,0.00,0.0,81.0,1.0,0.0,3.0,7.0,4.4,0.0,0.0,0.0,0.0,0.0,0.0,463.484166,198.153160,0.0,0.0,0.64,0.0,0.0,88.0,7.0,0.0,2.0,0.0,0.0,2.0,4.3,1.0,1.4,1.0,3.0,207.238037,206.536585,0.0,0.40,0.00,0.16,0.24,156.0,0.0,2.0,1
72797,sbPnQURi3j,A,49,U,7176.0,345,Samsung,Android OS,1,1,0,1,4G,SAATLI,9.0,2.0,103,138.5,2,0.9,2,0.9,0,0.0,36194.190184,36133.004917,0.0,0.10,2.80,0.10,0.0,302,103,2,5.00,1.0,80.0,86.0,4.0,5.7,1.0,4.1,3.0,1.6,13857.365302,13827.253128,0.0,0.50,1.70,0.25,0.25,292.0,80.0,4.0,5.00,1.0,74.0,129.5,2.0,2.1,1.0,0.4,1.0,1.8,0.000000,0.000000,0.0,0.15,1.70,0.05,0.10,238.0,74.0,2.0,3.0,62.0,121.0,7.0,6.7,1.0,0.6,6.0,6.1,0.000000,0.000000,0.0,0.65,2.500000e+00,0.05,0.6,239.0,62.0,7.0,5.0,80.0,202.6,3.0,3.9,0.0,0.0,3.0,3.9,0.000000,0.000000,0.0,0.5,2.35,0.0,0.5,219.0,80.0,3.0,5.0,87.0,110.6,4.0,2.7,0.0,0.0,4.0,2.7,0.000000,0.000000,0.0,0.25,2.35,0.00,0.25,241.0,87.0,4.0,1
68025,qOy-JlqPTl,M,49,M,4309.0,466,Samsung,Android OS,1,1,0,1,4G,BAKU,25.0,1.0,133,164.3,142,187.9,58,53.7,79,129.0,8181.310782,49.985725,0.0,0.00,0.00,0.00,0.0,411,133,142,25.00,1.0,209.0,273.0,193.0,192.0,61.0,49.1,126.0,124.3,7804.867967,65.904087,0.0,0.00,0.00,0.00,0.00,622.0,209.0,193.0,25.00,1.0,186.0,144.2,172.0,192.0,44.0,46.8,116.0,129.1,11567.962663,66.429559,0.0,0.00,0.00,0.00,0.00,542.0,186.0,172.0,25.0,115.0,130.7,97.0,124.5,29.0,34.6,68.0,89.8,8293.154886,44.728278,0.0,0.00,0.000000e+00,0.00,0.0,330.0,115.0,97.0,25.0,100.0,97.5,114.0,103.0,42.0,35.4,72.0,67.6,10192.908145,65.786654,0.0,0.0,0.00,0.0,0.0,307.0,100.0,114.0,25.0,142.0,121.3,80.0,60.6,29.0,21.0,51.0,39.6,9260.032768,51.885746,0.0,0.00,0.00,0.00,0.00,299.0,142.0,80.0,1
31781,MNODhcb7h0,A,50,U,6999.0,695,Samsung,Android OS,1,1,0,1,4G,ASTARA,9.0,3.0,102,85.1,1,0.1,1,0.1,0,0.0,1271.287395,1268.585261,0.0,0.05,0.55,0.05,0.0,169,102,1,10.00,4.0,117.0,97.6,1.0,1.0,1.0,1.0,0.0,0.0,106.041120,105.620465,0.0,0.05,0.00,0.05,0.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 9626 to 90423
Columns: 132 entries, telephone_number to target
dtypes: float64(111), int64(8), object(13)
memory usage: 10.1+ MB


In [6]:
print(df.columns.tolist())

['telephone_number', 'trf', 'age', 'gndr', 'tenure', 'age_dev', 'dev_man', 'device_os_name', 'dev_num', 'is_dualsim', 'is_featurephone', 'is_smartphone', 'simcard_type', 'region', 'val2_1', 'val3_1', 'val4_1', 'val5_1', 'val6_1', 'val7_1', 'val8_1', 'val9_1', 'val10_1', 'val11_1', 'val12_1', 'val13_1', 'val14_1', 'val15_1', 'val16_1', 'val17_1', 'val18_1', 'val19_1', 'val20_1', 'val21_1', 'val2_2', 'val3_2', 'val4_2', 'val5_2', 'val6_2', 'val7_2', 'val8_2', 'val9_2', 'val10_2', 'val11_2', 'val12_2', 'val13_2', 'val14_2', 'val15_2', 'val16_2', 'val17_2', 'val18_2', 'val19_2', 'val20_2', 'val21_2', 'val2_3', 'val3_3', 'val4_3', 'val5_3', 'val6_3', 'val7_3', 'val8_3', 'val9_3', 'val10_3', 'val11_3', 'val12_3', 'val13_3', 'val14_3', 'val15_3', 'val16_3', 'val17_3', 'val18_3', 'val19_3', 'val20_3', 'val21_3', 'val2_4', 'val4_4', 'val5_4', 'val6_4', 'val7_4', 'val8_4', 'val9_4', 'val10_4', 'val11_4', 'val12_4', 'val13_4', 'val14_4', 'val15_4', 'val16_4', 'val17_4', 'val18_4', 'val19_4', 'val

In [7]:
# Select only object columns
df_object = df.select_dtypes(include=['object'])

# Show info about them
df_object.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 9626 to 90423
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   telephone_number  10000 non-null  object
 1   trf               10000 non-null  object
 2   age               9976 non-null   object
 3   gndr              10000 non-null  object
 4   age_dev           9980 non-null   object
 5   dev_man           9980 non-null   object
 6   device_os_name    9980 non-null   object
 7   dev_num           9980 non-null   object
 8   is_dualsim        10000 non-null  object
 9   is_featurephone   10000 non-null  object
 10  is_smartphone     10000 non-null  object
 11  simcard_type      9651 non-null   object
 12  region            9904 non-null   object
dtypes: object(13)
memory usage: 1.1+ MB


In [8]:
from sklearn.feature_selection import mutual_info_regression
# Select features & target
X = df.drop(columns=['target'])
y = df['target']

# Encode object columns
X_encoded = X.copy()
for col in X_encoded.select_dtypes(include='object'):
    X_encoded[col] = X_encoded[col].astype('category').cat.codes

# Fill NaNs with a safe placeholder
X_encoded = X_encoded.fillna(-999)
y = y.fillna(y.mean())

mi_scores = mutual_info_regression(X_encoded, y, random_state=42)
mi_series = pd.Series(mi_scores, index=X_encoded.columns).sort_values(ascending=False)
print(mi_series.head(20))

val20_4    0.027031
val4_4     0.024791
val18_2    0.023587
val7_1     0.021939
val20_2    0.019226
val20_5    0.018787
val16_4    0.017719
val16_5    0.017012
trf        0.016835
val17_2    0.016686
val15_5    0.015876
region     0.014995
val9_6     0.014015
val15_4    0.013647
val7_2     0.013169
val14_6    0.012827
val14_2    0.012796
val5_2     0.012678
val16_1    0.012611
val15_2    0.011713
dtype: float64


Mutual Information was applied to identify which features are most related to the target, helping us select important features and reduce noise before modeling.


In [9]:
cat_cols = df.select_dtypes(include='object').columns.tolist()

# Temporary encoded dataframe
df_cat_encoded = df[cat_cols].apply(lambda x: x.astype('category').cat.codes)
df_cat_encoded['target'] = df['target']

In [10]:
from sklearn.feature_selection import mutual_info_regression

X_cat = df_cat_encoded.drop(columns=['target'])
y = df_cat_encoded['target']

mi_scores = mutual_info_regression(X_cat, y, discrete_features=True, random_state=42)
mi_series = pd.Series(mi_scores, index=X_cat.columns).sort_values(ascending=False)
print(mi_series)

age_dev             0.629722
region              0.021172
trf                 0.014260
dev_man             0.012172
simcard_type        0.008814
gndr                0.008529
device_os_name      0.005626
dev_num             0.005594
age                 0.003922
is_dualsim          0.003259
is_smartphone       0.002556
is_featurephone     0.002556
telephone_number    0.000000
dtype: float64


 As we can see, is_smartphone and is_featurephone columns have less importance. Thats why we drop them

In [11]:
df.set_index('telephone_number', inplace=True)
df = df.drop(['is_smartphone', 'is_featurephone'], axis=1)
df.head()

,trf,age,gndr,tenure,age_dev,dev_man,device_os_name,dev_num,is_dualsim,simcard_type,region,val2_1,val3_1,val4_1,val5_1,val6_1,val7_1,val8_1,val9_1,val10_1,val11_1,val12_1,val13_1,val14_1,val15_1,val16_1,val17_1,val18_1,val19_1,val20_1,val21_1,val2_2,val3_2,val4_2,val5_2,val6_2,val7_2,val8_2,val9_2,val10_2,val11_2,val12_2,val13_2,val14_2,val15_2,val16_2,val17_2,val18_2,val19_2,val20_2,val21_2,val2_3,val3_3,val4_3,val5_3,val6_3,val7_3,val8_3,val9_3,val10_3,val11_3,val12_3,val13_3,val14_3,val15_3,val16_3,val17_3,val18_3,val19_3,val20_3,val21_3,val2_4,val4_4,val5_4,val6_4,val7_4,val8_4,val9_4,val10_4,val11_4,val12_4,val13_4,val14_4,val15_4,val16_4,val17_4,val18_4,val19_4,val20_4,val21_4,val2_5,val4_5,val5_5,val6_5,val7_5,val8_5,val9_5,val10_5,val11_5,val12_5,val13_5,val14_5,val15_5,val16_5,val17_5,val18_5,val19_5,val20_5,val21_5,val2_6,val4_6,val5_6,val6_6,val7_6,val8_6,val9_6,val10_6,val11_6,val12_6,val13_6,val14_6,val15_6,val16_6,val17_6,val18_6,val19_6,val20_6,val21_6,target
telephone_number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
iWBn_13aJO,J,50,M,6957.0,1263,Samsung,Android OS,1,1,None,GAKH,3.0,3.0,14,7.4,0,0.0,0,0.0,0,0.0,0.000000,0.000000,0.0,0.00,1.12,0.00,0.0,79,14,0,3.00,3.0,13.0,9.0,1.0,0.4,1.0,0.4,0.0,0.0,0.000000,0.000000,0.0,0.14,1.05,0.14,0.00,64.0,13.0,1.0,3.00,3.0,14.0,10.3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.42,0.00,0.00,68.0,14.0,0.0,2.0,4.0,2.8,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,-1.490116e-09,0.00,0.0,39.0,4.0,0.0,3.0,10.0,8.3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.91,0.0,0.0,57.0,10.0,0.0,3.0,7.0,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.56,0.00,0.00,60.0,7.0,0.0,1
8JWaD2wYoQ,M,52,M,1041.0,125,Samsung,Android OS,2,1,4G,MASALLI,2.0,2.0,6,2.9,0,0.0,0,0.0,0,0.0,176.447991,0.000000,0.0,0.00,0.40,0.00,0.0,118,6,0,1.06,1.0,29.0,21.4,1.0,0.2,0.0,0.0,1.0,0.2,117.043976,116.052454,0.0,0.08,3.12,0.00,0.08,118.0,29.0,1.0,5.71,3.0,26.0,34.4,5.0,1.8,2.0,0.8,2.0,0.8,160.147566,67.178972,0.0,0.40,1.28,0.16,0.16,125.0,26.0,5.0,1.0,1.0,3.2,0.0,0.0,0.0,0.0,0.0,0.0,182.922203,181.783154,0.0,0.00,0.000000e+00,0.00,0.0,81.0,1.0,0.0,3.0,7.0,4.4,0.0,0.0,0.0,0.0,0.0,0.0,463.484166,198.153160,0.0,0.0,0.64,0.0,0.0,88.0,7.0,0.0,2.0,0.0,0.0,2.0,4.3,1.0,1.4,1.0,3.0,207.238037,206.536585,0.0,0.40,0.00,0.16,0.24,156.0,0.0,2.0,1
sbPnQURi3j,A,49,U,7176.0,345,Samsung,Android OS,1,1,4G,SAATLI,9.0,2.0,103,138.5,2,0.9,2,0.9,0,0.0,36194.190184,36133.004917,0.0,0.10,2.80,0.10,0.0,302,103,2,5.00,1.0,80.0,86.0,4.0,5.7,1.0,4.1,3.0,1.6,13857.365302,13827.253128,0.0,0.50,1.70,0.25,0.25,292.0,80.0,4.0,5.00,1.0,74.0,129.5,2.0,2.1,1.0,0.4,1.0,1.8,0.000000,0.000000,0.0,0.15,1.70,0.05,0.10,238.0,74.0,2.0,3.0,62.0,121.0,7.0,6.7,1.0,0.6,6.0,6.1,0.000000,0.000000,0.0,0.65,2.500000e+00,0.05,0.6,239.0,62.0,7.0,5.0,80.0,202.6,3.0,3.9,0.0,0.0,3.0,3.9,0.000000,0.000000,0.0,0.5,2.35,0.0,0.5,219.0,80.0,3.0,5.0,87.0,110.6,4.0,2.7,0.0,0.0,4.0,2.7,0.000000,0.000000,0.0,0.25,2.35,0.00,0.25,241.0,87.0,4.0,1
qOy-JlqPTl,M,49,M,4309.0,466,Samsung,Android OS,1,1,4G,BAKU,25.0,1.0,133,164.3,142,187.9,58,53.7,79,129.0,8181.310782,49.985725,0.0,0.00,0.00,0.00,0.0,411,133,142,25.00,1.0,209.0,273.0,193.0,192.0,61.0,49.1,126.0,124.3,7804.867967,65.904087,0.0,0.00,0.00,0.00,0.00,622.0,209.0,193.0,25.00,1.0,186.0,144.2,172.0,192.0,44.0,46.8,116.0,129.1,11567.962663,66.429559,0.0,0.00,0.00,0.00,0.00,542.0,186.0,172.0,25.0,115.0,130.7,97.0,124.5,29.0,34.6,68.0,89.8,8293.154886,44.728278,0.0,0.00,0.000000e+00,0.00,0.0,330.0,115.0,97.0,25.0,100.0,97.5,114.0,103.0,42.0,35.4,72.0,67.6,10192.908145,65.786654,0.0,0.0,0.00,0.0,0.0,307.0,100.0,114.0,25.0,142.0,121.3,80.0,60.6,29.0,21.0,51.0,39.6,9260.032768,51.885746,0.0,0.00,0.00,0.00,0.00,299.0,142.0,80.0,1
MNODhcb7h0,A,50,U,6999.0,695,Samsung,Android OS,1,1,4G,ASTARA,9.0,3.0,102,85.1,1,0.1,1,0.1,0,0.0,1271.287395,1268.585261,0.0,0.05,0.55,0.05,0.0,169,102,1,10.00,4.0,117.0,97.6,1.0,1.0,1.0,1.0,0

In [12]:
df_sample = df.sample(frac=0.2, random_state=42) # 20% sample (shuffled)
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, UdM8XYVVt9 to 0SqjCeBMFt
Columns: 129 entries, trf to target
dtypes: float64(111), int64(8), object(10)
memory usage: 2.0+ MB


In [13]:
X = df_sample.drop('target', axis = 1)
y = df_sample['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Identify columns by types
val_cols = [col for col in X_train.columns if col.startswith('val')]
numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.difference(val_cols)
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

PCA was applied to the val columns to reduce their high dimensionality and remove redundancy, making the model more efficient and less prone to overfitting.

In [15]:
val_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95)),
])

numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', CatBoostEncoder(handle_unknown='ignore')),
])

# combine all
preprocessor = ColumnTransformer([
    ('val', val_pipeline, val_cols),
    ('num', numeric_pipeline, numeric_cols),
    ('cat', categorical_pipeline, categorical_cols),
])

In [18]:
# Random Forest pipeline (classification)
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(random_state=42)),
])

# XGBoost pipeline (classification)
xgb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', XGBClassifier(random_state=42, eval_metric='logloss')),
])


In [20]:
# Cross-validation with accuracy
rf_scores = cross_val_score(rf_pipeline, X_train, y_train, cv=5, scoring='accuracy')
xgb_scores = cross_val_score(xgb_pipeline, X_train, y_train, cv=5, scoring='accuracy')

print(f"RF CV Accuracy: {np.mean(rf_scores):.4f} ± {np.std(rf_scores):.4f}")
print(f"XGB CV Accuracy: {np.mean(xgb_scores):.4f} ± {np.std(xgb_scores):.4f}")

# Cross-validation with weighted F1
rf_f1_scores = cross_val_score(rf_pipeline, X_train, y_train, cv=5, scoring='f1_weighted')
xgb_f1_scores = cross_val_score(xgb_pipeline, X_train, y_train, cv=5, scoring='f1_weighted')

print(f"RF CV F1 (weighted): {np.mean(rf_f1_scores):.4f} ± {np.std(rf_f1_scores):.4f}")
print(f"XGB CV F1 (weighted): {np.mean(xgb_f1_scores):.4f} ± {np.std(xgb_f1_scores):.4f}")

RF CV Accuracy: 0.5719 ± 0.0221
XGB CV Accuracy: 0.5669 ± 0.0318
RF CV F1 (weighted): 0.5688 ± 0.0239
XGB CV F1 (weighted): 0.5647 ± 0.0331


In [21]:
# Hypothesis testing (paired t-test)
stat, p_value = ttest_rel(rf_scores, xgb_scores)
print(f"Paired t-test p-value: {p_value:.4f}")

if p_value < 0.05:
    better_model = 'Random Forest' if np.mean(rf_scores) > np.mean(xgb_scores) else 'XGBoost'
    print(f"Statistically significant difference. Better model: {better_model}")
else:
    print("No statistically significant difference between models.")

Paired t-test p-value: 0.8279
No statistically significant difference between models.


In [22]:
# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 5),
        'use_label_encoder': False
    }

    # Build classifier
    model = XGBClassifier(
        random_state=42,
        eval_metric='logloss',
        **params
    )

    # Create pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model),
    ])

    # Evaluate with cross-validation (3-fold)
    scores = cross_val_score(pipeline, X_train, y_train, cv=3, scoring='accuracy')

    # We want to maximize accuracy
    return scores.mean()

# Run Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Print best results
print("Best trial:")
trial = study.best_trial
print(f"  Accuracy: {trial.value:.4f}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-08-13 22:19:02,329] A new study created in memory with name: no-name-a322a0c2-c009-42d6-a369-f433a3e7c1cd
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [22:19:02] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [22:19:03] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [22:19:04] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
[I 2025-08-13 22:19:04,730] Trial 0 finished with value: 0.5824942086931674 and parameters: {'n_estimators': 390, 'max_depth': 7, 'learning_rate': 0.1419947860556081, 'subsample': 0.9653561261059125, 'colsample_

Best trial:
  Accuracy: 0.5975
  Params:
    n_estimators: 183
    max_depth: 6
    learning_rate: 0.011836043445278416
    subsample: 0.8350286928223695
    colsample_bytree: 0.8460256447276271
    gamma: 2.739629633849611
    reg_alpha: 3.729788166892277
    reg_lambda: 3.5386814924579557


In [23]:
 optuna.visualization.plot_optimization_history(study).show()
 optuna.visualization.plot_param_importances(study).show()